In [5]:
# 這部分主要都是跑在 hightlight server , 這邊只是放sample

In [2]:
# 預處理, 建立index 檔案和 cache... ipython notebook 跑不動ＸＤＤ
# 全部都先放在 /work 資料夾當中

def create_file_name(data):
    md5 = hashlib.md5()
    for d in data:
        md5.update(d.encode('utf-8'))
    return md5.hexdigest()

def create_index_files(info, idxs, concat_path):
    video_path = "videos"
    name = create_file_name(info)
    target_concat_path = os.path.join(concat_path, name)

    f = open(target_concat_path, 'w+')
    for idx in idxs:
        idx = str(idx)
        url = "http://104.199.250.233:8000/material/scene/{}.m3u8".format(idx)
        output_mp4 = os.path.join(video_path, idx) + ".mp4"
        f.write('file /work/{}\n'.format(output_mp4))
        if os.path.exists(output_mp4):
            continue

        m3u8 = hls.M3U8.from_url(url)
        hls.download_to_mp4(m3u8, output_mp4)
    f.flush()
    f.close()


def run():
    for fn in ['event', 'team', 'player']:
        with open(fn) as f:
            for l in f.readlines():
                info = json.loads(l)
                create_index_files(info[:-1], info[-1], 'concat/{}'.format(fn))

In [3]:
# 球員錦集設定
# -*- coding: utf-8 -*-
from studio.models import PublishVideo
from django.core.files.base import ContentFile
import requests
from soccer.models import League
import json
import os
import hashlib
import requests
import re

dt = "player"

data = open('/work/{}'.format(dt)).readlines()


def create_file_name(data):
    md5 = hashlib.md5()
    for d in data:
        md5.update(d.encode('utf-8'))
    return md5.hexdigest()

docker_cmd = "sudo docker run -it --rm -v /work:/work -w /work/concat/{0} python ffmpeg -f concat -safe 0 -i {1} -vcodec copy {1}.mp4"
def setup_player_videos():
    l = League.objects.get(pk=111)
    for d in data:
        try:
            data = json.loads(d)
            video_name = data[0].replace(u'[全场]_', u'')
            player = data[2].replace(u'_score_player', u'')
            player_msg = u"【{}】全場精彩表現".format(player)

            file_name = create_file_name(data[:-1])
            idx = data[-1][0]

            v = PublishVideo.objects.create(name=u"{} {}".format(player_msg, video_name))

            m3u8 = "http://104.199.250.233:8000/material/scene/{}.m3u8".format(idx)
            content = requests.get(m3u8).content
            ts = re.findall(".*\.ts", content)
            preview = ts[0] + ".jpg"
            video_path = "/work/concat/{}/{}.mp4".format(dt, file_name)
            if not os.path.exists(video_path):
                os.system(docker_cmd.format(dt, file_name))

            print(v)
            with open(video_path) as f:
                mp4 = ContentFile(f.read())
            img = ContentFile(requests.get(preview).content)
            v.name = u"{} {}".format(player_msg, video_name)
            v.description = v.name
            v.cover_image.save('test.jpg', img)
            v.file.save('test.mp4', mp4)
            v.extra = data
            v.league = l
            v.save()
            os.system('sudo rm {}'.format(video_path))
        except Exception as e:
            import pdb;pdb.set_trace()
            print(e)
            
if __name__ == '__main__':
    setup_player_videos()
    
    

ModuleNotFoundError: No module named 'studio'

In [7]:
# 隊伍錦集 設定
# -*- coding: utf-8 -*-
from studio.models import PublishVideo
from django.core.files.base import ContentFile
import requests
from soccer.models import League
import json
import os
import hashlib
import requests
import re

dt = "team"

data = open('/work/{}'.format(dt)).readlines()

def create_file_name(data):
    md5 = hashlib.md5()
    for d in data:
        md5.update(d.encode('utf-8'))
    return md5.hexdigest()

l = League.objects.get(pk=111)
docker_cmd = "sudo docker run -it --rm -v /work:/work -w /work/concat/{0} python ffmpeg -f concat -safe 0 -i {1} -vcodec copy {1}.mp4"

def setup_team_videos():
    for d in data:
        try:
            data = json.loads(d)
            video_name = data[0].replace(u'[全场]_', u'')
            team = data[1].replace(u'_score', u'')
            team_msg = u"【{}】全场精华".format(team)

            file_name = create_file_name(data[:-1])
            idx = data[-1][0]

            v, _ = PublishVideo.objects.get_or_create(name=u"{} {}".format(team_msg, video_name))

            m3u8 = "http://104.199.250.233:8000/material/scene/{}.m3u8".format(idx)
            content = requests.get(m3u8).content
            ts = re.findall(".*\.ts", content)
            preview = ts[0] + ".jpg"
            video_path = "/work/concat/{}/{}.mp4".format(dt, file_name)
            if not os.path.exists(video_path):
                os.system(docker_cmd.format(dt, file_name))

            print(v)
            with open(video_path) as f:
                mp4 = ContentFile(f.read())
            img = ContentFile(requests.get(preview).content)
            v.name = u"{} {}".format(team_msg, video_name)
            v.description = v.name
            v.cover_image.save('test.jpg', img)
            v.file.save('test.mp4', mp4)
            v.extra = data
            v.league = l
            v.save()
            os.system('sudo rm {}'.format(video_path))
        except Exception as e:
            import pdb;pdb.set_trace()
            print(e)

if __name__ == '__main__':
    setup_team_videos()

ModuleNotFoundError: No module named 'studio'

In [8]:
# 事件錦集 處理
# -*- coding: utf-8 -*-
from studio.models import PublishVideo
from django.core.files.base import ContentFile
import requests
from soccer.models import League
import json
import os
import hashlib
import requests
import re

dbs = ['event', 'team', 'player']

data = open('/work/event').readlines()

def create_file_name(data):
    md5 = hashlib.md5()
    for d in data:
        md5.update(d.encode('utf-8'))
    return md5.hexdigest()

l = League.objects.get(pk=111)
docker_cmd = "sudo docker run -it --rm -v /work:/work -w /work/concat/{0} python ffmpeg -f concat -safe 0 -i {1} -vcodec copy {1}.mp4"

mapping = {
    u"三分_event": u"【三分球】本场三分球精华",
    u"灌篮_event": u"【灌篮】本场火热灌篮精华",
    u"快攻_event": u"【快攻】盘点本场精彩快攻",
    u"篮下_event": u"【禁区强打】本场禁区对抗精彩片段",
    u"射篮_event": u"【射篮】本场射篮精华",
}

def setup_event_videos():
    for d in data:
        try:
            data = json.loads(d)
            video_name = data[0].replace(u'[全场]_', u'')
            event = data[1]
            if not mapping.get(event, False):
                continue
            event_msg = mapping[event]

            file_name = create_file_name(data[:-1])
            idx = data[-1][0]

            v, _ = PublishVideo.objects.get_or_create(name=u"{} {}".format(event_msg, video_name))

            m3u8 = "http://104.199.250.233:8000/material/scene/{}.m3u8".format(idx)
            content = requests.get(m3u8).content
            ts = re.findall(".*\.ts", content)
            preview = ts[0] + ".jpg"
            video_path = "/work/concat/{}/{}.mp4".format("event", file_name)
            if not os.path.exists(video_path):
                os.system(docker_cmd.format("event", file_name))

            print(v)
            with open(video_path) as f:
                mp4 = ContentFile(f.read())
            img = ContentFile(requests.get(preview).content)
            v.name = u"{} {}".format(event_msg, video_name)
            v.cover_image.save('test.jpg', img)
            v.file.save('test.mp4', mp4)
            v.extra = data
            v.league = l
            v.save()
            os.system('sudo rm {}'.format(video_path))
        except Exception as e:
            print(e)

if __name__ == '__main__':
    setup_event_videos()

ModuleNotFoundError: No module named 'studio'

補充
---

In [6]:
# hls.py

import urllib
import requests
import re
import tempfile
import os
import copy
import json


class M3U8:
    def __init__(self, headers, scenes, base_url=""):
        self.headers = headers
        self.scenes = scenes
        self.base_url = base_url

    def slice(self, st, ed):
        # slice hls
        result = {}
        scenes = []
        for scene in copy.deepcopy(self.scenes):
            if scene['start'] + scene['duration'] > st and scene['start'] < ed:
                scenes.append(scene)

        headers = self.headers
        scenes = scenes
        return M3U8(headers, scenes, self.base_url)

    def select(self, idxs):
        idx = 0
        tmp = {}

        for scene in copy.deepcopy(self.scenes):
            if scene['idx'] in idxs:
                tmp[scene['idx']] = scene

        scenes = []
        new_idx = 0
        for idx in idxs:
            if tmp.get(idx, None):
                tmp[idx]['idx'] = new_idx
                scenes.append(tmp[idx])
                new_idx += 1
        return M3U8(self.headers, scenes, self.base_url)

    def render(self, clear=False, end=True):
        base_url = self.base_url
        context = ""
        context += self.headers
        if self.scenes:
            context = re.sub('#EXT-X-MEDIA-SEQUENCE:\d+', '#EXT-X-MEDIA-SEQUENCE:{}'.format(self.scenes[0]['idx']), context)

        scene_template = """
#EXTINF:{duration},{title}
{file_path}
        """.strip() + "\n"

        for scene in self.scenes:
            file_path = scene['file_path']
            if clear:
                file_path = file_path.split('/')[-1]

            scene['file_path'] = file_path
            context += scene_template.format(**scene)

        if end:
            context += "#EXT-X-ENDLIST"
        return context

    @staticmethod
    def from_url(url):
        resp = requests.get(url)
        return M3U8.from_context(resp.text, url)

    @staticmethod
    def from_path(path, base_url=""):
        with open(path) as f:
            context = f.read()
            return M3U8.from_context(context, base_url)

    @staticmethod
    def from_context(context, base_url=""):
        datas = re.split('(?=#EXTINF)', context)
        headers = datas[0]
        scenes = []

        # parse duration & title
        extinf = re.compile('^#EXTINF:([\d.]+),(.*)')

        # parse ts file path
        ts = re.compile('\n([^#].*)')

        raw_scenes = datas[1:]
        st = 0
        idx = 0
        for raw_scene in raw_scenes:
            duration, title = extinf.search(raw_scene).groups()
            file_path = ts.search(raw_scene).groups()[0]

            file_path = urllib.parse.urljoin(base_url, file_path)
            scene = {'title': title, 'duration': float(duration), "file_path": file_path, 'start': st, 'idx': idx}
            scenes.append(scene)
            st += scene['duration']
            idx += 1

        return M3U8.from_data(json.dumps({'headers': headers, 'scenes': scenes, 'base_url': base_url}))

    def create_m3u8_file(self, path):
        with open(path, 'w+') as f:
            f.write(self.render())
        return path

    def dump_data(self):
        data = {}
        data['headers'] = copy.deepcopy(self.headers)
        data['scenes'] = copy.deepcopy(self.scenes)
        data['base_url'] = copy.deepcopy(self.base_url)
        return json.dumps(data)

    @staticmethod
    def from_data(data):
        data = json.loads(data)
        return M3U8(data['headers'], data['scenes'], data['base_url'])


def create_path(path):
    try:
        os.makedirs(path)
    except:
        pass


def download(url, output_path):
    create_path(output_path)
    cmd = "ffmpeg -protocol_whitelist \"file,http,https,tcp,tls\" -i {url} -f hls {output_path}".format(url=url, output_path=output_path)
    os.system(cmd)
    return output_path


def download_to_mp4(m3u8, output_path):
    m3u8_path = urllib.parse.urljoin(output_path, "video.m3u8")
    m3u8.create_m3u8_file(m3u8_path)
    cmd = "ffmpeg -protocol_whitelist \"file,http,https,tcp,tls\" -i \"{m3u8_path}\" -c copy {output_path}".format(m3u8_path=m3u8_path, output_path=output_path)
    os.system(cmd)
    return output_path


def download_from_mp4(source_url, output_path):
    create_path(output_path)
    m3u8_path = os.path.join(output_path, 'video.m3u8')
    cmd = "ffmpeg -protocol_whitelist \"file,http,https,tcp,tls\" -i {} -f hls {}".format(source_url, m3u8_path)
    os.system(cmd)
    return output_path


def to_hls(source, dist):
    cmd = "ffmpeg -i {source}  -profile:v baseline -level 3.0 -s 640x360 -start_number 0 -hls_list_size 0 -f hls {dist}/video.m3u8".format(source=source, dist=dist)
    os.system(cmd)


def create_previews(hls_path, name_pattern="{}.jpg"):
    for root, folders, files in os.walk(hls_path):
        for f in files:
            filepath = os.path.join(root, f)
            if filepath.endswith('.ts'):
                cmd = "ffmpeg -i {} -vframes 1  {}".format(filepath, name_pattern.format(filepath))
                os.system(cmd)